# T-matrix computation for 3D obstacle scattering 主要是探究Grid function是否能建立

## Background

The infinite T-matrix can describes the acoustic scattering properties of an obstacle. In this tutorial, we will compute the T-matrix in a stable way. Consider the time-harmonic radiatiing acoustic field $u^{s}$ scattered by a three dimensional scatterer $\Omega$ in a homogeneous medium satifies the exterior Helmholtz equation: $$\Delta u^{s} + k^{2}u^{s} = 0, \ \text{in} \  \mathbb{R}^{2}\backslash\bar{\Omega},$$ where $k$ is the wavenumber and the Sommerfeld radiation condition $$\text{lim}_{|\boldsymbol{x}|\rightarrow\infty}\sqrt{|\boldsymbol{x}|}\left(\frac{\partial{u^{s}}}{\partial{\boldsymbol{x}}} - iku^{s}\right) = 0,$$ where the limit holds uniformly in all directions $\hat{\boldsymbol{x}} = \boldsymbol{x}\backslash|\boldsymbol{x}|$. This scattered field is induced by the incident field $u^{i}$ via one of the four boundary conditions which corresponding four properties of the scatterer: sound-soft, sound-hard, absorbing and TM-polarized dielectric.

We can expand the incident and scattered field in terms of the wave functions which satisfy the Helmholtz eqaution. 
$$u^{i}(\boldsymbol{x}) = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}p_{lj}\tilde{e_{lj}}(\boldsymbol{x}),$$
$$u^{s}(\boldsymbol{x}) = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}a_{lj}e_{lj}(\boldsymbol{x}),$$ 
where $$\tilde{e_{lj}}(\boldsymbol{x}) = j_{|l|}(k|\boldsymbol{x}|)Y_{lj}(\hat{\boldsymbol{x}})$$ is the regular spherical wave functions and 
$$e_{lj}(\boldsymbol{x}) = h_{|l|}^{(1)}(k|\boldsymbol{x}|)Y_{lj}(\hat{\boldsymbol{x}})$$ is the radiating spherical wave functions. Here, $J_{|n|}$ is the spherical Bessel function of degree $l$, $H_{|n|}^{(1)}$ is the spherical Hankel function of degree $l$, and $Y_{lj}$ is the spherical harmonic of degree $l$.

Since the Helmholtz equation is linear, there exist coefficients $t_{l'j',lj}$ such that $$a_{l'j'} = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}t_{l'j',lj}p_{lj}.$$ By writing $a = (a_{lj})$ and $p = (p_{lj})$, we define the (transition-) $T$-matrix as $T$ such that $a = Tp$. Our task is the compute this matrix using far field and the general steps are as follows.

The scattered field $u^{s}$ can be represented as $$u^{s} = Su^{i} = S\sum_{l = 0}^{\infty}\sum_{|j|\leq l}p_{lj}\tilde{e_{lj}}(\boldsymbol{x}) = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}p_{lj}S\tilde{e_{lj}}(\boldsymbol{x}) = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}p_{lj}u_{lj},$$ where $S$ is the operator that maps the incident field to its corresponding scattered field. For example, $S\tilde{e_{lj}} = u_{lj}$ is the scattered field induced by $\tilde{e_{lj}}$ and $$u_{lj} = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}t_{l'j',lj}e_{l'j'}$$. 

The far field of each $e_{l'j'}$ is $e_{l'j'}^{\infty} = \frac{1}{k}(-i)^{l'+1}Y_{l'j'}(\hat{\boldsymbol{x}})$, so we can write the far field of $u_{lj}$ as 
$$S^{\infty}\tilde{e_{lj}} = u^{\infty}_{lj} = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}t_{l'j',lj}e_{l'j'}^{\infty} = \sum_{l = 0}^{\infty}\sum_{|j|\leq l}t_{l'j',lj}\frac{1}{k}(-i)^{l'+1}Y_{l'j'}(\hat{\boldsymbol{x}})$$

By using the orthogonal property of $\text{exp}(in'\theta)$, the entries of the $T$-matrix can be expressed as 
$$t_{l'j',lj} = <u_{lj}^{\infty},Y_{l'j'}>.$$

We start with the usual imports.

In [17]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math

Define the wavenumber $k$, the 3D scatterer and function space.

In [18]:
k = 5

grid = bempp.api.shapes.regular_sphere(3)
vert_temp = grid.vertices
space = bempp.api.function_space(grid,'P',1)

Decide the index of the vertex whose first coordinate is zero, in order to avoid the 'divided by zero' situation and leave the valid vertices.

In [19]:
a = vert_temp[0,:]
ind = []
for i,j in enumerate(a):
    if j == 0:
        ind.append(i)
b = list(range(vert_temp.shape[1]))
for i in range(len(ind)):
    b.remove(ind[i])
vert = vert_temp[:,b]

Here, we define the identity, double layer and hypersingular boundary operators. We can use then to construct the LHS of the Burton-Miller formulation.

In [20]:
identity = bempp.api.operators.boundary.sparse.identity(space, space, space)
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space, space, space, k)
hyp = bempp.api.operators.boundary.helmholtz.hypersingular(space, space, space, k)

burton_miller = .5 * identity - dlp + (1j/k) * hyp

The following defines the spherical harmonic functions $Y_{lj}(\theta, \phi)$, spherical bessel function $j_{l}(k|x|)$ and construct $\tilde{e}_{lj}$.

In [21]:
def normalized_spherical_harmonics(p, q, x):
    azimuth = np.arctan(x[1]/x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    temp = np.array(scipy.special.lpmn(abs(p),q, np.cos(polar))[0])
    legd_poly = temp[-1,-1]
    return ((-1)**((p + abs(p)/2)))*np.sqrt(((2*q+1)/(4*np.pi))*(math.factorial(q-abs(p))/math.factorial(q+abs(p))))*legd_poly*np.exp(1j*p*azimuth)
    
def spherical_bessel_function(q,k,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,k,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,k,x)*normalized_spherical_harmonics(p,q,x)

In order to construct the normal derivative of the incident wave $\tilde{e}_{lj}(x)$, we need to define the derivative of the $j_{l}$ with regard to $|x|$, and $Y_{lj}$ with regard to $\theta$ and $\phi$.

In [22]:
def normalized_spherical_harmonics_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan(x[1]/x[0])
    y1 = normalized_spherical_harmonics(p, q, x)
    y2 = normalized_spherical_harmonics(abs(p + 1), q, x)
    return (abs(p) / np.tan(polar)) * y1 + np.sqrt((q - abs(p)) * (q + abs(p) + 1)) * np.exp(-1j * azimuth) * y2

def normalized_spherical_harmonics_dazimuth(p, q, x):
    return 1j * p * normalized_spherical_harmonics(p, q, x)

def spherical_bessel_function_dr(q,k,x):
    r = np.linalg.norm(x)
    return k*(spherical_bessel_function(q-1,k,x) - ((q+1)/(k*r))*spherical_bessel_function(q,k,x))

Neumann function and dirichlet function of $\tilde{e}_{lj}$ and use them to construct the grid functions.

In [24]:
@bempp.api.complex_callable
def dirichlet_fun(x,n,domain_index,result):
        with objmode():
            result[0] = regular_spherical_wavefunctions(p,q,k,x)
            
@bempp.api.complex_callable
def neumann_fun(x, n, domain_index, result):
    with objmode():
        r = np.linalg.norm(x)
        azimuth = np.arctan(x[1]/x[0])
        polar = np.arccos(x[2]/r)
        r_n = np.linalg.norm(n)
        
        grad_regular_sph_wf = [spherical_bessel_function_dr(q,k,x)*normalized_spherical_harmonics(p,q,x),
                              (1/r)*normalized_spherical_harmonics_dpolar(p,q,x)*spherical_bessel_function(q,k,x),
                              (1/(r*np.sin(polar)))*normalized_spherical_harmonics_dazimuth(p,q,x)*spherical_bessel_function(q,k,x)]
        normal_in_sph_coord = [r_n, np.arccos(n[2]/r_n), np.arctan(n[1]/n[0])]
        
        result[0] = np.inner(grad_regular_sph_wf, normal_in_sph_coord)
    


In [67]:
deg = 3

far_field_set = []
sph_points_set = []

for q in range(1,deg+1):
    for p in np.linspace(-q+1,q-1,2*q-1):
        dirichlet_grid_fun = bempp.api.GridFunction(space, fun = dirichlet_fun)
        neumann_grid_fun = bempp.api.GridFunction(space, fun = neumann_fun)

        rhs_fun = dirichlet_grid_fun + (1j/k)*neumann_grid_fun

        total_field, info, it_count = bempp.api.linalg.gmres(burton_miller,rhs_fun,use_strong_form=True, return_iteration_count=True)
        dlp_far_field = bempp.api.operators.far_field.helmholtz.double_layer(space,vert,k)
        far_field_set.append(dlp_far_field * total_field)

        sph_points = np.zeros(vert.shape[1], dtype = complex)
        for i in range(vert.shape[1]):
            sph_points[i] = normalized_spherical_harmonics(p,q,vert[:,i])
        sph_points_set.append(sph_points)

/home/xiaoshu/.conda/envs/opencl/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [70]:
deg = 3

for q in range(1,deg+1):
    for p in np.linspace(-q+1,q-1,2*q-1):
        dirichlet_grid_fun = bempp.api.GridFunction(space, fun = dirichlet_fun)
        neumann_grid_fun = bempp.api.GridFunction(space, fun = neumann_fun)

        rhs_fun = dirichlet_grid_fun + (1j/k)*neumann_grid_fun

        total_field, info, it_count = bempp.api.linalg.gmres(burton_miller,rhs_fun,use_strong_form=True, return_iteration_count=True)
        dlp_far_field = bempp.api.operators.far_field.helmholtz.double_layer(space,vert,k)
        print(dlp_far_field*total_field)


/home/xiaoshu/.conda/envs/opencl/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


[[nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj
  nan+nanj nan+nanj nan+nanj nan+nanj 

In [30]:
far_field.shape

(1, 226)

In [16]:
vert.shape[1]

226

In [55]:
np.linspace(-1+1,1-1,2*1-1)

array([0.])